## DATASET AND HELPER FUNCTIONS

In [ ]:
import pandas as pd

# Load your CSV file
df = pd.read_csv("/content/malicious_phish.csv")

# Print unique classes to verify
print("Original classes:", df['type'].unique())

# Convert multi-class labels to binary
df['label'] = df['type'].apply(lambda x: 'safe' if x.lower() == 'benign' else 'unsafe')

# Drop the old column if you no longer need it
df.drop(columns=['type'], inplace=True)

# Save new version
df.to_csv("binary_url_safe_urls_train.csv", index=False)

# Check new label distribution
print(df['label'].value_counts())


Original classes: ['phishing' 'benign' 'defacement' 'malware']
label
safe      428103
unsafe    223088
Name: count, dtype: int64


In [ ]:
pip install pybloom_live

In [ ]:

import pandas as pd
import numpy as np
from pybloom_live import BloomFilter, ScalableBloomFilter
import time
import sys
from sklearn.model_selection import train_test_split
import psutil
import os
from tqdm import tqdm
import gc
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import roc_curve

# PyTorch imports for GPU support
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from collections import defaultdict
import numpy as np
import math



In [ ]:

class PyTorchLogisticRegression(nn.Module):
    """
    PyTorch implementation of Logistic Regression that can use GPU if available.
    """
    def __init__(self, input_dim):
        super(PyTorchLogisticRegression, self).__init__()
        self.linear = nn.Linear(input_dim, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        out = self.linear(x)
        out = self.sigmoid(out)
        return out

In [ ]:
class PyTorchRandomForest(nn.Module):
    """
    PyTorch implementation of a neural network that approximates a Random Forest.
    This is a simplified version with more hidden units to capture complex patterns.
    """
    def __init__(self, input_dim):
        super(PyTorchRandomForest, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(input_dim, 256),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.model(x)

In [ ]:
def preprocess_url(url):
    """Extract features from URL for ML model."""
    # Basic preprocessing - convert to lowercase and remove punctuation
    url = url.lower()
    url = re.sub(r'[^\w\s]', '', url)

    # Extract simple features
    features = {
        'length': len(url),
        'num_digits': sum(c.isdigit() for c in url),
        'num_segments': len(url.split('/')),
    }

    return url, features

## STANDARD BLOOM FILTER

In [ ]:
def evaluate_standard_bloom_filter(train_urls, test_urls, capacity, error_rate):
    """
    Evaluate a standard Bloom filter.

    Args:
        train_urls: URLs to add to the filter
        test_urls: URLs to test against the filter
        capacity: Expected number of elements
        error_rate: Desired false positive rate

    Returns:
        Dictionary with evaluation results
    """
    print(f"\nEvaluating Standard Bloom Filter with error_rate={error_rate}")

    # Initialize memory and time tracking
    start_time = time.time()

    # Calculate the size of the Bloom filter in bits
    filter_bits = int(-capacity * np.log(error_rate) / (np.log(2) ** 2))
    filter_size_mb = filter_bits / 8 / 1024 
    print(f"Calculated Bloom filter size: {filter_size_mb:.2f} KB ({filter_bits} bits)")

    # Create Bloom filter
    bloom = BloomFilter(capacity=capacity, error_rate=error_rate)

    creation_time = time.time() - start_time

    # Add all training URLs to the filter
    print(f"Adding {len(train_urls)} URLs to the filter...")
    add_start_time = time.time()
    progress_bar = tqdm(total=len(train_urls), desc="Adding URLs", unit="url")

    for url in train_urls["url"]:
        bloom.add(url)
        progress_bar.update(1)
    progress_bar.close()
    add_time = time.time() - add_start_time
    print(f"Added {len(train_urls)} URLs in {add_time:.2f} seconds")

    # Test on the test_urls
    print(f"Testing {len(test_urls)} URLs against the filter...")
    test_start_time = time.time()

    true_positives = 0
    false_positives = 0
    true_negatives = 0
    false_negatives = 0

    membership_check = list(train_urls['url'])

    progress_bar = tqdm(total=len(test_urls), desc="Testing URLs", unit="url")
    for url in test_urls["url"]:
        if url in bloom:
            if url in membership_check:
                true_positives += 1
            else:
                false_positives += 1
        else:
            if url in membership_check:
                false_negatives += 1
            else:
                true_negatives += 1
        progress_bar.update(1)
    progress_bar.close()
    test_time = time.time() - test_start_time
    print(f"Tested {len(test_urls)} URLs in {test_time:.2f} seconds")

    # Calculate metrics
    false_positive_rate = false_positives / len(test_urls) if len(test_urls) > 0 else 0
    false_negative_rate = false_negatives / len(train_urls) if len(train_urls) > 0 else 0
    bits_per_item = filter_bits / len(train_urls) if len(train_urls) > 0 else 0

    # Prepare results
    results = {
        "filter_type": f"Standard Bloom Filter (error_rate={error_rate})",
        "error_rate": error_rate,
        "capacity": capacity,
        "filter_size_bits": filter_bits,
        "filter_size_kb": filter_size_mb,
        "bits_per_item": bits_per_item,
        "creation_time": creation_time,
        "add_time": add_time,
        "true_positives": true_positives,
        "false_negatives": false_negatives,
        "true_negatives": true_negatives,
        "false_positives": false_positives,
        "false_positive_rate": false_positive_rate,
        "false_negative_rate": false_negative_rate,
        "test_time": test_time,
        "accuracy": (true_positives + true_negatives) / len(test_urls) if len(test_urls) > 0 else 0,
        "precision": true_positives / (true_positives + false_positives) if (true_positives + false_positives) > 0 else 0,
        "recall": true_positives / (true_positives + false_negatives) if (true_positives + false_negatives) > 0 else 0,
        "f1_score": (2 * true_positives) / (2 * true_positives + false_positives + false_negatives) if (true_positives + false_positives + false_negatives) > 0 else 0,
    }


    # Print summary
    print("\nResults:")
    print(f"  Filter type: {results['filter_type']}")
    # print(f"  Capacity: {results['capacity']}")
    print(f"  Filter size: {results['filter_size_kb']:.4f} KB")
    print(f"  Bits per item: {results['bits_per_item']:.2f}")
    # print(f"  Creation time: {results['creation_time']:.2f} seconds")
    # print(f"  Add time: {results['add_time']:.2f} seconds")
    print(f"  Query time: {results['test_time']:.2f} seconds")
    print(f"  True positives: {results['true_positives']}")
    print(f"  False positives: {results['false_positives']}")
    print(f"  True negatives: {results['true_negatives']}")
    print(f"  False negatives: {results['false_negatives']}")
    print(f"  False positive rate: {results['false_positive_rate']:.4f} (expected: {results['error_rate']})")
    print(f"  False negative rate: {results['false_negative_rate']:.4f}")
    print(f"  Accuracy: {results['accuracy']:.4f}")
    print(f"  Precision: {results['precision']:.4f}")
    print(f"  Recall: {results['recall']:.4f}")
    print(f"  F1 Score: {results['f1_score']:.4f}")

    return results

## PARTITIONED LEARNED BLOOM FILTER

In [ ]:
class PartitionedLearnedBloomFilter:
    """
    Partitioned Learned Bloom Filter implementation.

    This filter uses a machine learning model to partition the input space
    and assigns different Bloom filters to different partitions.
    """

    def __init__(self, capacity, error_rate=0.01, num_partitions=3):
        """
        Initialize the Partitioned Learned Bloom Filter.

        Args:
            capacity: Expected number of elements
            error_rate: Desired false positive rate
            num_partitions: Number of partitions to create
        """
        self.capacity = capacity
        self.error_rate = error_rate
        self.num_partitions = num_partitions

        # Check if CUDA is available
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        print(f"Using device: {self.device}")

        # Initialize ML model
        self.model = None
        self.pytorch_model = None
        self.vectorizer = TfidfVectorizer(analyzer='char', ngram_range=(3, 5))

        # Thresholds for partitioning
        self.thresholds = []

        # Initialize Bloom filters for each partition
        # We'll adjust capacities based on expected elements per partition
        self.bloom_filters = []
        self.partition_capacities = []

        # Statistics
        self.items_added = 0
        self.items_per_partition = [0] * num_partitions

    def train_model(self, train_urls, train_labels):
        """
        Train the ML model for partitioning.

        Args:
            train_urls: List of URLs for training
            train_labels: Binary labels (0 for safe, 1 for unsafe)
        """
        print("Training ML model for partitioning...")

        # First, fit the vectorizer to transform text data
        print("Fitting vectorizer...")
        X = self.vectorizer.fit_transform(train_urls)

        # Convert labels to numpy array if they're not already
        train_labels = np.array(train_labels)

        # For CPU fallback, use scikit-learn pipeline
        if self.device.type == 'cpu':
            print("Using CPU with scikit-learn pipeline")
            self.model = Pipeline([
                ('classifier', LogisticRegression(max_iter=1000))
            ])

            print("Fitting model to training data...")
            self.model.fit(X, train_labels)

            # Get predicted probabilities
            print("Getting predicted probabilities...")
            # Create a progress bar for prediction
            probs = np.zeros(len(train_urls))
            progress_bar = tqdm(total=len(train_urls), desc="Predicting probabilities", unit="url")

            # Process in batches to show progress
            batch_size = 1000
            for i in range(0, len(train_urls), batch_size):
                batch_end = min(i + batch_size, len(train_urls))
                batch_vectors = X[i:batch_end]
                batch_probs = self.model.predict_proba(batch_vectors)[:, 1]
                probs[i:batch_end] = batch_probs
                progress_bar.update((batch_vectors.shape[0]))

            progress_bar.close()
        else:
            # For GPU, use PyTorch model
            print("Using GPU with PyTorch model")

            # Convert to PyTorch tensors
            X_tensor = torch.FloatTensor(X.toarray()).to(self.device)
            y_tensor = torch.FloatTensor(train_labels.reshape(-1, 1)).to(self.device)

            # Create PyTorch model
            input_dim = X.shape[1]
            self.pytorch_model = PyTorchLogisticRegression(input_dim).to(self.device)

            # Define loss function and optimizer
            criterion = nn.BCELoss()
            optimizer = optim.Adam(self.pytorch_model.parameters(), lr=0.01)

            # Create DataLoader for batch processing
            dataset = TensorDataset(X_tensor, y_tensor)
            dataloader = DataLoader(dataset, batch_size=128, shuffle=True)

            # Train the model
            print("Training PyTorch model...")
            num_epochs = 10
            for epoch in range(num_epochs):
                running_loss = 0.0
                progress_bar = tqdm(dataloader, desc=f"Epoch {epoch+1}/{num_epochs}", leave=False)

                for inputs, labels in progress_bar:
                    # Zero the parameter gradients
                    optimizer.zero_grad()

                    # Forward pass
                    outputs = self.pytorch_model(inputs)
                    loss = criterion(outputs, labels)

                    # Backward pass and optimize
                    loss.backward()
                    optimizer.step()

                    # Update statistics
                    running_loss += loss.item() * inputs.size(0)
                    progress_bar.set_postfix({"loss": f"{loss.item():.4f}"})

                epoch_loss = running_loss / len(dataset)
                print(f"Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}")

            # Set model to evaluation mode
            self.pytorch_model.eval()

            # Get predicted probabilities
            print("Getting predicted probabilities...")
            unsafe_train_urls = [url for url, label in zip(train_urls, train_labels) if label == 1]

            probs = np.zeros(len(unsafe_train_urls))
            progress_bar = tqdm(total=len(unsafe_train_urls), desc="Predicting probabilities", unit="url")

            # Process in batches to show progress
            batch_size = 1000
            with torch.no_grad():
                for i in range(0, len(unsafe_train_urls), batch_size):
                    batch_end = min(i + batch_size, len(unsafe_train_urls))
                    batch_vectors = X[i:batch_end].toarray()
                    batch_tensor = torch.FloatTensor(batch_vectors).to(self.device)
                    batch_probs = self.pytorch_model(batch_tensor).cpu().numpy().flatten()
                    probs[i:batch_end] = batch_probs
                    progress_bar.update((batch_vectors.shape[0]))

            progress_bar.close()

        # Compute thresholds for partitioning
        self.thresholds = []
        for i in range(1, self.num_partitions):
            threshold = np.percentile(probs, (i * 100) / self.num_partitions)
            self.thresholds.append(threshold)

        # Count expected elements per partition
        partition_counts = [0] * self.num_partitions
        progress_bar = tqdm(total=len(probs), desc="Calculating partition counts", unit="item")

        # Process in batches for better performance
        batch_size = 10000
        for i in range(0, len(probs), batch_size):
            batch_end = min(i + batch_size, len(probs))
            for prob in probs[i:batch_end]:
                partition = self._get_partition(prob)
                partition_counts[partition] += 1
            progress_bar.update(batch_end - i)

        progress_bar.close()

        # Calculate capacity for each partition
        # Add some buffer to each partition to avoid 100% fill rates
        total_size = 0
        total_count = sum(partition_counts)
        self.partition_capacities = [
            max(int((count / total_count) * self.capacity * 1.2), 1)  # Add 20% buffer
            for count in partition_counts
        ]

        # Initialize ScalableBloomFilters with adjusted initial capacities
        self.bloom_filters = [
            ScalableBloomFilter(initial_capacity=cap, error_rate=self.error_rate)
            for cap in self.partition_capacities
        ]

        print(f"Model trained. Partition capacities: {self.partition_capacities}")

    def _get_partition(self, probability):
        """
        Determine which partition an element belongs to based on its probability.

        Args:
            probability: ML model's predicted probability

        Returns:
            Partition index (0 to num_partitions-1)
        """
        for i, threshold in enumerate(self.thresholds):
            if probability < threshold:
                return i
        return self.num_partitions - 1

    def _get_probability(self, url):
        """
        Get the probability from the model for a single URL.

        Args:
            url: URL to get probability for

        Returns:
            Probability value
        """
        if self.device.type == 'cpu' or self.pytorch_model is None:
            # Use scikit-learn model
            url_vector = self.vectorizer.transform([url])
            return self.model.predict_proba(url_vector)[0, 1]
        else:
            # Use PyTorch model
            url_vector = self.vectorizer.transform([url]).toarray()
            url_tensor = torch.FloatTensor(url_vector).to(self.device)

            # Get prediction
            with torch.no_grad():
                prob = self.pytorch_model(url_tensor).cpu().numpy()[0, 0]

            return prob

    def add(self, url):
        """
        Add a URL to the filter.

        Args:
            url: URL to add
        """
        if (self.device.type == 'cpu' and self.model is None) or \
           (self.device.type == 'cuda' and self.pytorch_model is None):
            raise ValueError("Model must be trained before adding elements")

        # Get probability from model
        prob = self._get_probability(url)

        # Determine partition
        partition = self._get_partition(prob)

        # Add to corresponding Bloom filter
        self.bloom_filters[partition].add(url)

        # Update statistics
        self.items_added += 1
        self.items_per_partition[partition] += 1

    def query(self, url):
        """
        Query if a URL is in the filter.

        Args:
            url: URL to query

        Returns:
            True if the URL might be in the filter, False if definitely not
        """
        if (self.device.type == 'cpu' and self.model is None) or \
           (self.device.type == 'cuda' and self.pytorch_model is None):
            raise ValueError("Model must be trained before querying")

        # Get probability from model
        prob = self._get_probability(url)

        # Determine partition
        partition = self._get_partition(prob)

        # Query corresponding Bloom filter
        return url in self.bloom_filters[partition]

    def get_stats(self):
        """Return statistics about the filter."""
        # Calculate total bits used by all Bloom filters
        total_bits = 0
        bloom_filter_sizes_mb = []

        for bf in self.bloom_filters:
            # ScalableBloomFilter doesn't have a bitarray attribute
            # Instead, it has a series of BloomFilter objects in its 'filters' attribute
            if hasattr(bf, 'filters'):
                # For ScalableBloomFilter
                bf_bits = sum(len(f.bitarray) for f in bf.filters)
                bf_size_mb = bf_bits / 8 / 1024 / 1024  # Convert to MB
            else:
                # For regular BloomFilter (fallback)
                bf_bits = len(bf.bitarray)
                bf_size_mb = bf_bits / 8 / 1024 / 1024  # Convert to MB

            total_bits += bf_bits
            bloom_filter_sizes_mb.append(bf_size_mb)

        total_size_mb = total_bits / 8 / 1024 / 1024  # Convert bits to MB

        # Calculate bits per item
        bits_per_item = total_bits / self.items_added if self.items_added > 0 else 0

        stats = {
            "total_items": self.items_added,
            "items_per_partition": self.items_per_partition,
            "partition_capacities": self.partition_capacities,
            "partition_fill_rates": [
                self.items_per_partition[i] / self.partition_capacities[i]
                for i in range(self.num_partitions)
            ],
            "num_partitions": self.num_partitions,
            "error_rate": self.error_rate,
            "total_bits": total_bits,
            "total_size_mb": total_size_mb,
            "bits_per_item": bits_per_item,
            "bloom_filter_sizes_mb": bloom_filter_sizes_mb
        }
        return stats

def get_memory_usage():
    """Return the current memory usage in MB."""
    # Force garbage collection before measuring memory
    gc.collect()
    process = psutil.Process(os.getpid())
    return process.memory_info().rss / (1024 * 1024)


In [ ]:

def evaluate_partitioned_learned_bloom_filter(train_urls, train_labels, test_urls, test_labels, capacity, error_rate, num_partitions):
    """
    Evaluate a Partitioned Learned Bloom Filter.

    Args:
        train_urls: URLs for training
        train_labels: Labels for training (0 for safe, 1 for unsafe)
        test_urls: URLs for testing
        test_labels: Labels for testing
        capacity: Expected number of elements
        error_rate: Desired false positive rate
        num_partitions: Number of partitions

    Returns:
        Dictionary with evaluation results
    """
    print(f"\nEvaluating Partitioned Learned Bloom Filter with {num_partitions} partitions and error_rate={error_rate}")

    # Initialize memory and time tracking
    start_time = time.time()
    memory_before = get_memory_usage()
    print(f"Initial memory usage: {memory_before:.2f} MB")

    # Create and train PLBF
    plbf = PartitionedLearnedBloomFilter(capacity, error_rate, num_partitions)
    plbf.train_model(train_urls, train_labels)

    # Track memory after model creation
    memory_after_model = get_memory_usage()
    print(f"Memory after model creation: {memory_after_model:.2f} MB")
    model_creation_time = time.time() - start_time
    unsafe_train_urls = [url for url, label in zip(train_urls, train_labels) if label == 1]

    # Add all training URLs to the filter
    print(f"Adding {len(unsafe_train_urls)} URLs to the filter...")
    add_start_time = time.time()
    progress_bar = tqdm(total=len(unsafe_train_urls), desc="Adding URLs", unit="url")
    
    # Process in batches for better progress tracking
    batch_size = 1000
    for i in range(0, len(unsafe_train_urls), batch_size):
        batch_end = min(i + batch_size, len(unsafe_train_urls))
        for url in unsafe_train_urls[i:batch_end]:
            plbf.add(url)
        progress_bar.update(batch_end - i)

    progress_bar.close()

    add_time = time.time() - add_start_time
    memory_after_add = get_memory_usage()
    print(f"Memory after adding URLs: {memory_after_add:.2f} MB")
    # Test on the test_urls
    print(f"Testing {len(test_urls)} URLs against the filter...")
    test_start_time = time.time()

    true_positives = 0
    false_positives = 0
    true_negatives = 0
    false_negatives = 0

    progress_bar = tqdm(total=len(test_urls), desc="Testing URLs", unit="url")
    for url, label in zip(test_urls, test_labels):
        result = plbf.query(url)  # Predicts: unsafe (True) or safe (False)

        if result:  # Bloom filter says: possibly unsafe
            if label == 1:  # Actually unsafe
                true_positives += 1
            else:           # Actually safe
                false_positives += 1
        else:  # Bloom filter says: definitely safe
            if label == 1:  # Actually unsafe
                false_negatives += 1
            else:           # Actually safe
                true_negatives += 1
        progress_bar.update(1)
    progress_bar.close()
    test_time = time.time() - test_start_time
    print(f"Tested {len(test_urls)} URLs in {test_time:.2f} seconds")

    # Calculate metrics
    false_positive_rate = false_positives / len(test_urls) if len(test_urls) > 0 else 0
    false_negative_rate = false_negatives / len(train_urls) if len(train_urls) > 0 else 0
    # bits_per_item = filter_bits / len(train_urls) if len(train_urls) > 0 else 0

    # Get filter stats
    filter_stats = plbf.get_stats()

    # Prepare results
    results = {
        "filter_type": f"PLBF (partitions={num_partitions}, error_rate={error_rate})",
        "num_partitions": num_partitions,
        "error_rate": error_rate,
        "capacity": capacity,
        "memory_before": memory_before,
        "memory_after_model": memory_after_model,
        "memory_after_add": memory_after_add,
        "memory_for_model": memory_after_model - memory_before,
        "memory_for_filters": filter_stats["total_size_mb"],
        "total_memory": (memory_after_model - memory_before) + filter_stats["total_size_mb"],
        "model_creation_time": model_creation_time,
        "add_time": add_time,
        "items_per_partition": filter_stats["items_per_partition"],
        "partition_fill_rates": filter_stats["partition_fill_rates"],
        "bits_per_item": filter_stats["bits_per_item"],
        "total_bits": filter_stats["total_bits"],
        "capacity": capacity,
        "true_positives": true_positives,
        "false_negatives": false_negatives,
        "true_negatives": true_negatives,
        "false_positives": false_positives,
        "false_positive_rate": false_positive_rate,
        "false_negative_rate": false_negative_rate,
        "test_time": test_time,
        "accuracy": (true_positives + true_negatives) / len(test_urls) if len(test_urls) > 0 else 0,
        "precision": true_positives / (true_positives + false_positives) if (true_positives + false_positives) > 0 else 0,
        "recall": true_positives / (true_positives + false_negatives) if (true_positives + false_negatives) > 0 else 0,
        "f1_score": (2 * true_positives) / (2 * true_positives + false_positives + false_negatives) if (true_positives + false_positives + false_negatives) > 0 else 0,
    }


    # Print summary
    print("\nResults:")
    print(f"  Filter type: {results['filter_type']}")
   
    print(f"  Memory for model: {results['memory_for_model']:.2f} MB")
    print(f"  Memory for Bloom filters: {results['memory_for_filters']:.2f} MB")
    print(f"  Total memory (model + filters): {results['total_memory']:.2f} MB")
    print(f"  Bits per item stats: {results['bits_per_item']:.2f}")
    # print(f"  Creation time: {results['creation_time']:.2f} seconds")
    # print(f"  Add time: {results['add_time']:.2f} seconds")
    print(f"  Query time: {results['test_time']:.2f} seconds")
    print(f"  Items per partition: {filter_stats['items_per_partition']}")
    print(f"  Partition fill rates: {[f'{rate:.2f}' for rate in filter_stats['partition_fill_rates']]}")

    print(f"  True positives: {results['true_positives']}")
    print(f"  False positives: {results['false_positives']}")
    print(f"  True negatives: {results['true_negatives']}")
    print(f"  False negatives: {results['false_negatives']}")
    print(f"  False positive rate: {results['false_positive_rate']:.4f} (expected: {results['error_rate']})")
    print(f"  False negative rate: {results['false_negative_rate']:.4f}")
    print(f"  Accuracy: {results['accuracy']:.4f}")
    print(f"  Precision: {results['precision']:.4f}")
    print(f"  Recall: {results['recall']:.4f}")
    print(f"  F1 Score: {results['f1_score']:.4f}")

    return results
    
    



## CASCADED LEARNED BLOOM FILTER

In [ ]:
class CascadedLearnedBloomFilter:
    """
    Cascaded Learned Bloom Filter implementation.

    This filter uses a cascade of machine learning models and Bloom filters
    to achieve optimal model-filter size balance and fast rejection.
    The final stage includes a safety net Bloom filter to catch any false negatives.
    """

    def __init__(self, capacity, error_rate=0.01, num_stages=2, thresholds=None):
        """
        Initialize the Cascaded Learned Bloom Filter.

        Args:
            capacity: Expected number of elements
            error_rate: Desired false positive rate
            num_stages: Number of cascade stages
            thresholds: List of thresholds for each stage (if None, will be determined automatically)
        """
        self.capacity = capacity
        self.error_rate = error_rate
        self.num_stages = num_stages

        # Check if CUDA is available
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        print(f"Using device: {self.device}")

        # Initialize ML models for each stage
        self.models = []
        self.pytorch_models = []
        self.vectorizers = []
        for _ in range(num_stages):
            self.vectorizers.append(TfidfVectorizer(analyzer='char', ngram_range=(3, 5)))
            self.models.append(None)
            self.pytorch_models.append(None)

        # Thresholds for each stage
        if thresholds is None:
            # Default thresholds will be determined during training
            self.thresholds = [0.5] * num_stages
        else:
            self.thresholds = thresholds

        # Initialize Bloom filters for each stage
        self.bloom_filters = []

        # Statistics
        self.items_added = 0
        self.stage_stats = [{
            'items_processed': 0,
            'items_rejected': 0,
            'items_passed': 0
        } for _ in range(num_stages)]
        
        # Track false negatives for final safety net Bloom filter
        self.false_negatives_detected = 0

    def _create_model(self, stage):
        """Create a model for a specific stage."""
        if self.device.type == 'cpu':
            # CPU-based models using scikit-learn
            if stage == 0:
                # First stage uses a simpler model for faster processing
                return Pipeline([
                    ('vectorizer', self.vectorizers[stage]),
                    ('classifier', LogisticRegression(max_iter=1000))
                ])
            else:
                # Later stages use more complex models for better accuracy
                return Pipeline([
                    ('vectorizer', self.vectorizers[stage]),
                    ('classifier', RandomForestClassifier(n_estimators=100))
                ])
        else:
            # GPU-based models using PyTorch
            # We'll create the actual PyTorch models during training
            # This method just returns a placeholder for scikit-learn compatibility
            if stage == 0:
                # First stage uses a simpler model for faster processing
                return Pipeline([
                    ('vectorizer', self.vectorizers[stage]),
                    ('classifier', LogisticRegression(max_iter=1))
                ])
            else:
                # Later stages use more complex models for better accuracy
                return Pipeline([
                    ('vectorizer', self.vectorizers[stage]),
                    ('classifier', RandomForestClassifier(n_estimators=100))
                ])

    def train_models(self, train_urls, train_labels):
        """
        Train the cascade of ML models.

        Args:
            train_urls: List of URLs for training
            train_labels: Binary labels (0 for safe, 1 for unsafe)
        """
        print("Training cascade of ML models...")

        # Convert to numpy arrays if they're not already
        train_urls = np.array(train_urls)
        train_labels = np.array(train_labels)

        # Keep track of remaining data for each stage
        remaining_urls = train_urls
        remaining_labels = train_labels
        
        # Keep track of URLs that pass the final stage but are actually malicious
        final_stage_false_negatives = []

        # Train each stage
        for stage in range(self.num_stages):
            print(f"Training stage {stage+1}/{self.num_stages}...")

            # Create and train the model for this stage
            self.models[stage] = self._create_model(stage)

            # First, fit the vectorizer to transform text data
            print(f"Fitting vectorizer for stage {stage+1}...")
            X = self.vectorizers[stage].fit_transform(remaining_urls)

            if self.device.type == 'cpu':
                # CPU training with scikit-learn
                print(f"Fitting model for stage {stage+1} using CPU...")
                # For scikit-learn, we need to fit the classifier part of the pipeline
                self.models[stage].named_steps['classifier'].fit(X, remaining_labels)

                # Get predicted probabilities
                print(f"Getting predicted probabilities for stage {stage+1}...")
                probs = np.zeros(len(remaining_urls))
                progress_bar = tqdm(total=len(remaining_urls), desc=f"Predicting probabilities (stage {stage+1})", unit="url")

                # Process in batches to show progress
                batch_size = 1000
                for i in range(0, len(remaining_urls), batch_size):
                    batch_end = min(i + batch_size, len(remaining_urls))
                    batch_vectors = X[i:batch_end]
                    batch_probs = self.models[stage].named_steps['classifier'].predict_proba(batch_vectors)[:, 1]
                    probs[i:batch_end] = batch_probs
                    progress_bar.update((batch_vectors.shape[0]))

                progress_bar.close()
            else:
                # GPU training with PyTorch
                print(f"Fitting model for stage {stage+1} using GPU...")

                # Convert to PyTorch tensors
                X_tensor = torch.FloatTensor(X.toarray()).to(self.device)
                y_tensor = torch.FloatTensor(remaining_labels.reshape(-1, 1)).to(self.device)

                # Create PyTorch model based on stage
                input_dim = X.shape[1]
                if stage == 0:
                    # First stage uses a simpler model for faster processing
                    self.pytorch_models[stage] = PyTorchLogisticRegression(input_dim).to(self.device)
                else:
                    # Later stages use more complex models for better accuracy
                    self.pytorch_models[stage] = PyTorchRandomForest(input_dim).to(self.device)

                # Define loss function and optimizer
                criterion = nn.BCELoss()
                optimizer = optim.Adam(self.pytorch_models[stage].parameters(), lr=0.01)

                # Create DataLoader for batch processing
                dataset = TensorDataset(X_tensor, y_tensor)
                dataloader = DataLoader(dataset, batch_size=128, shuffle=True)

                # Train the model
                print(f"Training PyTorch model for stage {stage+1}...")
                num_epochs = 1
                for epoch in range(num_epochs):
                    running_loss = 0.0
                    progress_bar = tqdm(dataloader, desc=f"Epoch {epoch+1}/{num_epochs}", leave=False)

                    for inputs, labels in progress_bar:
                        # Zero the parameter gradients
                        optimizer.zero_grad()

                        # Forward pass
                        outputs = self.pytorch_models[stage](inputs)
                        loss = criterion(outputs, labels)

                        # Backward pass and optimize
                        loss.backward()
                        optimizer.step()

                        # Update statistics
                        running_loss += loss.item() * inputs.size(0)
                        progress_bar.set_postfix({"loss": f"{loss.item():.4f}"})

                    epoch_loss = running_loss / len(dataset)
                    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}")

                # Set model to evaluation mode
                self.pytorch_models[stage].eval()

                # Get predicted probabilities
                print(f"Getting predicted probabilities for stage {stage+1}...")
                probs = np.zeros(len(remaining_urls))
                progress_bar = tqdm(total=len(remaining_urls), desc=f"Predicting probabilities (stage {stage+1})", unit="url")

                # Process in batches to show progress
                batch_size = 1000
                with torch.no_grad():
                    for i in range(0, len(remaining_urls), batch_size):
                        batch_end = min(i + batch_size, len(remaining_urls))
                        batch_vectors = X[i:batch_end].toarray()
                        batch_tensor = torch.FloatTensor(batch_vectors).to(self.device)
                        batch_probs = self.pytorch_models[stage](batch_tensor).cpu().numpy().flatten()
                        probs[i:batch_end] = batch_probs
                        progress_bar.update((batch_vectors.shape[0]))

                progress_bar.close()

            # Find optimal threshold using ROC curve
            fpr, tpr, thresholds = roc_curve(remaining_labels, probs)

            # Find threshold that gives desired false positive rate
            target_fpr = self.error_rate / self.num_stages  # Distribute error rate across stages
            idx = np.argmin(np.abs(fpr - target_fpr))
            self.thresholds[stage] = thresholds[idx]

            print(f"Optimal threshold for stage {stage+1}: {self.thresholds[stage]:.4f}")

            # Identify items that pass this stage (high probability)
            passed_mask = probs >= self.thresholds[stage]
            passed_urls = remaining_urls[passed_mask]
            passed_labels = remaining_labels[passed_mask]

            # Handle false negatives (items that are actually malicious but were classified as benign)
            # For all stages, we need to track these
            false_negative_mask = (~passed_mask) & (remaining_labels == 1)
            false_negative_urls = remaining_urls[false_negative_mask]
            
            # Create Bloom filter for this stage
            if len(false_negative_urls) > 0:
                stage_error_rate = self.error_rate / self.num_stages
                # Use a reasonable initial capacity but allow growth
                initial_capacity = max(len(false_negative_urls), 100)
                bloom = ScalableBloomFilter(initial_capacity=initial_capacity, error_rate=stage_error_rate)

                # Add false negative items to Bloom filter
                print(f"Adding {len(false_negative_urls)} false negatives to Bloom filter for stage {stage+1}...")
                progress_bar = tqdm(total=len(false_negative_urls), desc=f"Adding to Bloom filter (stage {stage+1})", unit="url")

                for url in false_negative_urls:
                    bloom.add(url)
                    progress_bar.update(1)

                progress_bar.close()

                self.bloom_filters.append(bloom)
                
                # If this is the final stage, these are our final false negatives
                if stage == self.num_stages - 1:
                    final_stage_false_negatives = false_negative_urls
            else:
                # No false negatives to add to Bloom filter
                self.bloom_filters.append(None)

            # Special handling for the final stage
            if stage == self.num_stages - 1:
                # Track stats for final stage false negatives
                print(f"Final stage false negatives detected: {len(final_stage_false_negatives)}")
                self.false_negatives_detected = len(final_stage_false_negatives)
                
                # Final stage also needs to catch predicted safe but actually malicious URLs
                final_false_negative_mask = passed_mask & (remaining_labels == 1)
                final_false_negative_urls = remaining_urls[final_false_negative_mask]
                
                if len(final_false_negative_urls) > 0:
                    print(f"Adding {len(final_false_negative_urls)} additional malicious URLs to final safety net Bloom filter...")
                    
                    # Create additional safety net Bloom filter if needed
                    if self.bloom_filters[-1] is None:
                        # Create new Bloom filter for safety net
                        safety_net_error_rate = self.error_rate / 2  # Stricter error rate for safety net
                        safety_net_capacity = max(len(final_false_negative_urls), 100)
                        safety_net_bloom = ScalableBloomFilter(initial_capacity=safety_net_capacity, error_rate=safety_net_error_rate)
                        self.bloom_filters[-1] = safety_net_bloom
                    
                    # Add URLs to the safety net Bloom filter
                    progress_bar = tqdm(total=len(final_false_negative_urls), desc="Adding to safety net Bloom filter", unit="url")
                    for url in final_false_negative_urls:
                        self.bloom_filters[-1].add(url)
                        progress_bar.update(1)
                    progress_bar.close()

            # Update remaining data for next stage
            if stage < self.num_stages - 1:
                remaining_urls = passed_urls
                remaining_labels = passed_labels
                print(f"Items passing to stage {stage+2}: {len(remaining_urls)} ({len(remaining_urls)/len(train_urls)*100:.2f}% of original data)")

                if len(remaining_urls) == 0:
                    print(f"No items left for stage {stage+2}. Stopping cascade training.")
                    break

        print("Cascade training complete.")

    def _get_probability(self, url, stage):
        """
        Get the probability from the model for a single URL at a specific stage.

        Args:
            url: URL to get probability for
            stage: Stage index

        Returns:
            Probability value
        """
        if self.device.type == 'cpu' or self.pytorch_models[stage] is None:
            # Use scikit-learn model
            url_vector = self.vectorizers[stage].transform([url])
            return self.models[stage].named_steps['classifier'].predict_proba(url_vector)[:, 1][0]
        else:
            # Use PyTorch model
            url_vector = self.vectorizers[stage].transform([url]).toarray()
            url_tensor = torch.FloatTensor(url_vector).to(self.device)

            # Get prediction
            with torch.no_grad():
                prob = self.pytorch_models[stage](url_tensor).cpu().numpy()[0, 0]

            return prob

    def add(self, url):
        """
        Add a URL to the filter.

        Args:
            url: URL to add
        """
        if (self.device.type == 'cpu' and not self.models[0]) or \
           (self.device.type == 'cuda' and not self.pytorch_models[0]):
            raise ValueError("Models must be trained before adding elements")

        # Process through the cascade
        for stage in range(self.num_stages):
            # Get probability from model
            prob = self._get_probability(url, stage)

            # Update statistics
            self.stage_stats[stage]['items_processed'] += 1

            if prob < self.thresholds[stage]:
                # Item rejected by this stage's model
                self.stage_stats[stage]['items_rejected'] += 1

                # Add to Bloom filter if this is a known malicious URL
                if self.bloom_filters[stage]:
                    self.bloom_filters[stage].add(url)

                break
            else:
                # Item passed this stage
                self.stage_stats[stage]['items_passed'] += 1

                # If this is the last stage, we're done
                if stage == self.num_stages - 1:
                    # For the final stage, we add known malicious URLs to the safety net Bloom filter
                    if self.bloom_filters[stage]:
                        self.bloom_filters[stage].add(url)
                    break

        # Update total items added
        self.items_added += 1

    def query(self, url):
        """
        Query if a URL is potentially malicious.

        Args:
            url: URL to query

        Returns:
            True if the URL might be malicious, False if definitely benign
        """
        if (self.device.type == 'cpu' and not self.models[0]) or \
           (self.device.type == 'cuda' and not self.pytorch_models[0]):
            raise ValueError("Models must be trained before querying")

        # Process through the cascade
        for stage in range(self.num_stages):
            # Get probability from model
            prob = self._get_probability(url, stage)

            if prob < self.thresholds[stage]:
                # Item rejected by this stage's model
                
                # Check Bloom filter for this stage to see if it's a false negative
                if self.bloom_filters[stage] and url in self.bloom_filters[stage]:
                    # URL is in the Bloom filter, so it's potentially malicious
                    return True
                else:
                    # URL is not in the Bloom filter, so it's definitely benign
                    return False
            else:
                # Item passed this stage, continue to next stage
                if stage == self.num_stages - 1:
                    # Final stage classifier says it's malicious
                    return True
                    
                    # Note: We don't need to check the final Bloom filter here
                    # because if the final classifier says it's malicious, we
                    # trust that decision. The final Bloom filter is only used
                    # to catch false negatives that the classifier thought were benign.

        # Should never reach here
        return False

    def get_stats(self):
        """Return statistics about the filter."""
        # Calculate total bits used by all Bloom filters
        total_bits = 0
        bloom_filter_sizes = []

        for bf in self.bloom_filters:
            if bf:
                # ScalableBloomFilter doesn't have a bitarray attribute
                # Instead, it has a series of BloomFilter objects in its 'filters' attribute
                if hasattr(bf, 'filters'):
                    # For ScalableBloomFilter
                    bf_bits = sum(len(f.bitarray) for f in bf.filters)
                else:
                    # For regular BloomFilter (fallback)
                    bf_bits = len(bf.bitarray)

                total_bits += bf_bits
                bloom_filter_sizes.append(bf_bits / 8 / 1024 / 1024)  # Convert to MB
            else:
                bloom_filter_sizes.append(0)

        total_size_mb = total_bits / 8 / 1024 / 1024  # Convert bits to MB

        # Calculate bits per item
        bits_per_item = total_bits / self.items_added if self.items_added > 0 else 0

        stats = {
            "total_items": self.items_added,
            "num_stages": self.num_stages,
            "thresholds": self.thresholds,
            "stage_stats": self.stage_stats,
            "error_rate": self.error_rate,
            "total_bits": total_bits,
            "total_size_mb": total_size_mb,
            "bits_per_item": bits_per_item,
            "bloom_filter_sizes_mb": bloom_filter_sizes,
            "false_negatives_detected": self.false_negatives_detected
        }
        return stats

In [ ]:
def evaluate_cascaded_learned_bloom_filter(train_urls, train_labels, test_urls, test_labels, capacity, error_rate=0.01, num_stages=2):
    """
    Evaluate a Cascaded Learned Bloom Filter.

    Args:
        train_urls: URLs for training
        train_labels: Labels for training (0 for safe, 1 for unsafe)
        test_urls: URLs for testing
        test_labels: Labels for testing (not used in current implementation but included for future label-specific analysis)
        capacity: Expected number of elements
        error_rate: Desired false positive rate
        num_stages: Number of cascade stages

    Returns:
        Dictionary with evaluation results
    """

    print(f"\nEvaluating Cascaded Learned Bloom Filter with {num_stages} stages and error_rate={error_rate}")

    # Initialize memory and time tracking
    start_time = time.time()
    memory_before = get_memory_usage()
    print(f"Initial memory usage: {memory_before:.2f} MB")

    # Create and train CLBF
    clbf = CascadedLearnedBloomFilter(capacity, error_rate, num_stages)
    clbf.train_models(train_urls, train_labels)

    # Track memory after model creation
    memory_after_model = get_memory_usage()
    print(f"Memory after model creation: {memory_after_model:.2f} MB")
    model_creation_time = time.time() - start_time

    unsafe_train_urls = [url for url, label in zip(train_urls, train_labels) if label == 1]
    # Add all training URLs to the filter

    print(f"Adding {len(unsafe_train_urls)} URLs to the filter...")
    add_start_time = time.time()
    progress_bar = tqdm(total=len(unsafe_train_urls), desc="Adding URLs", unit="url")
    
    # Process in batches for better progress tracking
    batch_size = 1000
    for i in range(0, len(unsafe_train_urls), batch_size):
        batch_end = min(i + batch_size, len(unsafe_train_urls))
        for url in unsafe_train_urls[i:batch_end]:
            clbf.add(url)
        progress_bar.update(batch_end - i)

    progress_bar.close()
    

    progress_bar.close()

    add_time = time.time() - add_start_time
    memory_after_add = get_memory_usage()
    print(f"Memory after adding URLs: {memory_after_add:.2f} MB")

        # Test on the test_urls
    print(f"Testing {len(test_urls)} URLs against the filter...")
    test_start_time = time.time()

    true_positives = 0
    false_positives = 0
    true_negatives = 0
    false_negatives = 0

    progress_bar = tqdm(total=len(test_urls), desc="Testing URLs", unit="url")
    for url, label in zip(test_urls, test_labels):
        result = clbf.query(url)  # Predicts: unsafe (True) or safe (False)

        if result:  # Bloom filter says: possibly unsafe
            if label == 1:  # Actually unsafe
                true_positives += 1
            else:           # Actually safe
                false_positives += 1
        else:  # Bloom filter says: definitely safe
            if label == 1:  # Actually unsafe
                false_negatives += 1
            else:           # Actually safe
                true_negatives += 1
        progress_bar.update(1)
    progress_bar.close()
    test_time = time.time() - test_start_time
    print(f"Tested {len(test_urls)} URLs in {test_time:.2f} seconds")

    # Calculate metrics
    false_positive_rate = false_positives / len(test_urls) if len(test_urls) > 0 else 0
    false_negative_rate = false_negatives / len(train_urls) if len(train_urls) > 0 else 0
    # bits_per_item = filter_bits / len(train_urls) if len(train_urls) > 0 else 0

    # Get filter stats
    filter_stats = clbf.get_stats()

    # Prepare results
    results = {
        "filter_type": f"CLBF (stages={num_stages}, error_rate={error_rate})",
        "num_stages": num_stages,
        "error_rate": error_rate,
        "capacity": capacity,
        "memory_before": memory_before,
        "memory_after_model": memory_after_model,
        "memory_after_add": memory_after_add,
        "memory_for_model": memory_after_model - memory_before,
        "memory_for_filters": filter_stats["total_size_mb"],
        "total_memory": (memory_after_model - memory_before) + filter_stats["total_size_mb"],
        "model_creation_time": model_creation_time,
        "add_time": add_time,
        "true_positives": true_positives,
        "false_negatives": false_negatives,
        "true_negatives": true_negatives,
        "false_positives": false_positives,
        "false_positive_rate": false_positive_rate,
        "false_negative_rate": false_negative_rate,
        "stage_stats": filter_stats["stage_stats"],
        "thresholds": filter_stats["thresholds"],
        "bits_per_item": filter_stats["bits_per_item"],
        "test_time": test_time,
        "total_bits": filter_stats["total_bits"],
        "accuracy": (true_positives + true_negatives) / len(test_urls) if len(test_urls) > 0 else 0,
        "precision": true_positives / (true_positives + false_positives) if (true_positives + false_positives) > 0 else 0,
        "recall": true_positives / (true_positives + false_negatives) if (true_positives + false_negatives) > 0 else 0,
        "f1_score": (2 * true_positives) / (2 * true_positives + false_positives + false_negatives) if (true_positives + false_positives + false_negatives) > 0 else 0,
    
    }

    # Print summary
    print("\nResults:")
    print(f"  False Positive Rate: {false_positive_rate:.4f} (expected: {error_rate})")
    print(f"  False Negative Rate: {false_negative_rate:.4f}")
    print(f"  Memory for model: {results['memory_for_model']:.2f} MB")
    print(f"  Memory for Bloom filters: {results['memory_for_filters']:.2f} MB")
    print(f"  Total memory (model + filters): {results['total_memory']:.2f} MB")
    print(f"  Bits per item: {results['bits_per_item']:.2f}")
    print(f"  Model creation time: {model_creation_time:.2f} seconds")
    print(f"  Add time: {add_time:.2f} seconds")
    print(f"  Test time: {results['test_time']} seconds")
    print(f"  True positives: {results['true_positives']}")
    print(f"  False positives: {results['false_positives']}")
    print(f"  True negatives: {results['true_negatives']}")
    print(f"  False negatives: {results['false_negatives']}")
    print(f"  False positive rate: {results['false_positive_rate']:.4f} (expected: {results['error_rate']})")
    print(f"  False negative rate: {results['false_negative_rate']:.4f}")
    print(f"  Accuracy: {results['accuracy']:.4f}")
    print(f"  Precision: {results['precision']:.4f}")
    print(f"  Recall: {results['recall']:.4f}")
    print(f"  F1 Score: {results['f1_score']:.4f}")
    print(f"  Stage statistics:")
    for stage in range(num_stages):
        stats = filter_stats["stage_stats"][stage]
        print(f"    Stage {stage+1}: Processed {stats['items_processed']}, "
              f"Rejected {stats['items_rejected']} ({stats['items_rejected']/stats['items_processed']*100 if stats['items_processed'] > 0 else 0:.2f}%), "
              f"Passed {stats['items_passed']} ({stats['items_passed']/stats['items_processed']*100 if stats['items_processed'] > 0 else 0:.2f}%)")
    print(f"  Bloom filter sizes: {[f'{size:.2f} MB' for size in filter_stats['bloom_filter_sizes_mb']]}")

    return results


## DMLBF

In [ ]:

class EntropyCalculator:
    """Calculates and tracks entropy for data partitions."""
    
    def __init__(self):
        self.partition_histograms = defaultdict(lambda: defaultdict(int))
        self.partition_counts = defaultdict(int)
        self.partition_entropies = {}
    
    def update(self, partition_id, element_hash):
        """Update histograms with new element."""
        # Use a binned hash value to track distribution
        binned_hash = element_hash % 1000  # Create 1000 bins for histogram
        self.partition_histograms[partition_id][binned_hash] += 1
        self.partition_counts[partition_id] += 1
        # Recalculate entropy for this partition
        self._calculate_entropy(partition_id)
    
    def _calculate_entropy(self, partition_id):
        """Calculate Shannon entropy for a partition."""
        histogram = self.partition_histograms[partition_id]
        total_count = self.partition_counts[partition_id]
        
        if total_count == 0:
            self.partition_entropies[partition_id] = 0
            return
        
        entropy = 0
        for count in histogram.values():
            probability = count / total_count
            if probability > 0:
                entropy -= probability * math.log2(probability)
        
        self.partition_entropies[partition_id] = entropy
    
    def get_entropy(self, partition_id):
        """Get entropy for a specific partition."""
        return self.partition_entropies.get(partition_id, 0)
    
    def get_all_entropies(self):
        """Get entropies for all partitions."""
        return self.partition_entropies

class AdaptivePartitioningModel(nn.Module):
    """Neural network that learns to assign elements to partitions based on features."""
    
    def __init__(self, input_dim, num_partitions):
        super(AdaptivePartitioningModel, self).__init__()
        self.num_partitions = num_partitions
        
        # Feature extraction layers
        self.feature_layers = nn.Sequential(
            nn.Linear(input_dim, 256),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Dropout(0.2)
        )
        
        # Partition assignment layer
        self.partition_layer = nn.Linear(128, num_partitions)
        
        # Confidence layer (for entropy-aware thresholding)
        self.confidence_layer = nn.Linear(128, 1)
        
        self.softmax = nn.Softmax(dim=1)
        self.sigmoid = nn.Sigmoid()
    
    def forward(self, x):
        features = self.feature_layers(x)
        partition_logits = self.partition_layer(features)
        partition_probs = self.softmax(partition_logits)
        confidence = self.sigmoid(self.confidence_layer(features))
        
        return partition_probs, confidence

class CascadingBloomFilter:
    """Multi-level Bloom filter with cascading structure."""
    
    def __init__(self, initial_capacity, error_rate, num_levels=3):
        self.num_levels = num_levels
        self.filters = []
        
        # Create a cascade of increasingly precise Bloom filters
        for level in range(num_levels):
            # Each level has lower error rate but smaller capacity
            level_error = error_rate / (level + 1)
            level_capacity = initial_capacity // (2 ** level)
            
            self.filters.append(
                ScalableBloomFilter(
                    initial_capacity=max(level_capacity, 100),
                    error_rate=level_error
                )
            )
        
        self.items_added = 0
    
    def add(self, element):
        """Add element to all levels of the cascade."""
        for filter_level in self.filters:
            filter_level.add(element)
        self.items_added += 1
    
    def check(self, element, max_level=None):
        """Check if element exists in the cascade, up to max_level."""
        if max_level is None:
            max_level = self.num_levels
        
        # Check each level in sequence
        for level in range(min(max_level, self.num_levels)):
            if element not in self.filters[level]:
                return False
        
        return True
    
    def get_size_bits(self):
        """Get total size in bits."""
        total_bits = 0
        for filter_level in self.filters:
            if hasattr(filter_level, 'filters'):
                # For ScalableBloomFilter
                level_bits = sum(len(f.bitarray) for f in filter_level.filters)
            else:
                # For regular BloomFilter
                level_bits = len(filter_level.bitarray)
            total_bits += level_bits
        
        return total_bits

class DynamicMultiLevelLearnedBloomFilter:
    """
    Dynamic Multi-Level Learned Bloom Filter with Entropy Checking (DML-LBF)
    
    Combines learned models, partitioning, cascading, adaptivity, and entropy checking
    into a unified system for efficient membership testing.
    """
    
    def __init__(self, expected_capacity, error_rate=0.01, initial_partitions=8):
        """
        Initialize the DML-LBF.
        
        Args:
            expected_capacity: Expected number of elements
            error_rate: Target false positive rate
            initial_partitions: Initial number of partitions
        """
        self.expected_capacity = expected_capacity
        self.error_rate = error_rate
        self.num_partitions = initial_partitions
        
        # Check if CUDA is available
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        print(f"Using device: {self.device}")
        
        # Text vectorizer for feature extraction
        self.vectorizer = TfidfVectorizer(analyzer='char', ngram_range=(3, 5))
        
        # Partitioning model (will be initialized during training)
        self.model = None
        
        # Entropy tracker
        self.entropy_calculator = EntropyCalculator()
        
        # Partition Bloom filters
        self.partition_filters = {}
        
        # Partition metadata
        self.partition_stats = defaultdict(lambda: {
            'items_added': 0,
            'queries': 0,
            'hits': 0,
            'last_accessed': time.time()
        })
        
        # System-wide statistics
        self.total_items = 0
        self.total_queries = 0
        self.maintenance_counter = 0
        
        # Adaptive thresholds
        self.entropy_threshold_high = 4.0  # High entropy threshold for merging
        self.entropy_threshold_low = 2.0   # Low entropy threshold for splitting
        self.query_threshold = 1000        # Minimum queries before adaptation
        self.maintenance_interval = 10000  # Items before maintenance check
        
        # Partition mapping (for when partitions are merged/split)
        self.partition_mapping = {}  # Maps old partition IDs to new ones
    
    def _initialize_partition_filter(self, partition_id, capacity=None):
        """Initialize a new cascading Bloom filter for a partition."""
        if capacity is None:
            capacity = max(self.expected_capacity // self.num_partitions, 100)
        
        self.partition_filters[partition_id] = CascadingBloomFilter(
            initial_capacity=capacity,
            error_rate=self.error_rate,
            num_levels=3
        )
    
    def train(self, training_data, labels=None):
        """
        Train the partitioning model and initialize the system.
        
        Args:
            training_data: List of elements for training
            labels: Optional binary labels (1 for membership, 0 for non-membership)
        """
        print("Training DML-LBF...")
        
        # Step 1: Vectorize the training data
        print("Vectorizing training data...")
        X = self.vectorizer.fit_transform(training_data)
        input_dim = X.shape[1]
        
        # Step 2: Initialize and train the partitioning model
        print("Training partitioning model...")
        self.model = AdaptivePartitioningModel(input_dim, self.num_partitions).to(self.device)
        
        # Convert to PyTorch tensors
        X_tensor = torch.FloatTensor(X.toarray()).to(self.device)
        
        # If labels are provided, use them for supervised training
        if labels is not None:
            y_tensor = torch.FloatTensor(labels.reshape(-1, 1)).to(self.device)
            dataset = TensorDataset(X_tensor, y_tensor)
        else:
            # For unsupervised training, use dummy labels
            dummy_labels = torch.zeros(X_tensor.shape[0], 1).to(self.device)
            dataset = TensorDataset(X_tensor, dummy_labels)
        
        # Train the model
        dataloader = DataLoader(dataset, batch_size=128, shuffle=True)
        criterion = nn.CrossEntropyLoss()
        optimizer = optim.Adam(self.model.parameters(), lr=0.001)
        
        num_epochs = 5
        for epoch in range(num_epochs):
            running_loss = 0.0
            progress_bar = tqdm(dataloader, desc=f"Epoch {epoch+1}/{num_epochs}")
            
            for inputs, _ in progress_bar:
                optimizer.zero_grad()
                
                # Forward pass
                partition_probs, _ = self.model(inputs)
                
                # For unsupervised learning, we want to maximize entropy of partition assignments
                # This encourages balanced partitioning
                partition_entropy = -torch.sum(partition_probs * torch.log(partition_probs + 1e-10), dim=1).mean()
                loss = -partition_entropy  # Negative because we want to maximize entropy
                
                # Add regularization to prevent empty partitions
                partition_counts = torch.sum(partition_probs, dim=0)
                balance_penalty = torch.var(partition_counts) * 0.1
                loss += balance_penalty
                
                # Backward pass and optimize
                loss.backward()
                optimizer.step()
                
                running_loss += loss.item()
                progress_bar.set_postfix({"loss": f"{loss.item():.4f}"})
            
            print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(dataloader):.4f}")
        
        # Step 3: Determine initial partition assignments
        print("Determining initial partition assignments...")
        partition_assignments = self._get_partition_assignments(X_tensor)
        
        # Step 4: Initialize Bloom filters for each partition
        print("Initializing Bloom filters for each partition...")
        partition_counts = defaultdict(int)
        for partition_id in partition_assignments:
            partition_counts[partition_id] += 1
        
        for partition_id in range(self.num_partitions):
            # Estimate capacity based on assignments
            capacity = max(partition_counts.get(partition_id, 0) * 2, 100)
            self._initialize_partition_filter(partition_id, capacity)
        
        # Step 5: Add training elements to appropriate partitions
        print("Adding training elements to partitions...")
        progress_bar = tqdm(total=len(training_data), desc="Adding elements")
        
        for i, element in enumerate(training_data):
            partition_id = partition_assignments[i]
            
            # Add to appropriate Bloom filter
            self.partition_filters[partition_id].add(element)
            
            # Update entropy calculation
            element_hash = hash(element)
            self.entropy_calculator.update(partition_id, element_hash)
            
            # Update statistics
            self.partition_stats[partition_id]['items_added'] += 1
            self.total_items += 1
            
            progress_bar.update(1)
        
        progress_bar.close()
        
        # Step 6: Calculate initial entropy for each partition
        print("Calculating initial entropy for each partition...")
        entropies = self.entropy_calculator.get_all_entropies()
        for partition_id, entropy in entropies.items():
            print(f"Partition {partition_id}: Entropy = {entropy:.4f}, Items = {self.partition_stats[partition_id]['items_added']}")
        
        print("DML-LBF training complete.")
    
    def _get_partition_assignments(self, X_tensor):
        """Get partition assignments for a batch of elements."""
        self.model.eval()
        with torch.no_grad():
            partition_probs, _ = self.model(X_tensor)
            partition_assignments = torch.argmax(partition_probs, dim=1).cpu().numpy()
        return partition_assignments
    
    def _get_partition_probs(self, element):
        """Get partition probabilities for a single element."""
        # Vectorize the element
        X = self.vectorizer.transform([element]).toarray()
        X_tensor = torch.FloatTensor(X).to(self.device)
        
        # Get predictions
        self.model.eval()
        with torch.no_grad():
            partition_probs, confidence = self.model(X_tensor)
            
            # Convert to numpy for easier handling
            partition_probs = partition_probs.cpu().numpy()[0]
            confidence = confidence.cpu().numpy()[0][0]
        
        return partition_probs, confidence
    
    def _get_relevant_partitions(self, element, threshold=0.1):
        """
        Get relevant partitions for an element based on model predictions.
        
        Returns partitions sorted by probability (highest first).
        """
        partition_probs, confidence = self._get_partition_probs(element)
        
        # Adjust threshold based on confidence
        adaptive_threshold = threshold * (1 - confidence * 0.5)
        
        # Get partitions with probability above threshold
        relevant_partitions = []
        for partition_id, prob in enumerate(partition_probs):
            if prob >= adaptive_threshold:
                # Include entropy in the sorting criteria
                entropy = self.entropy_calculator.get_entropy(partition_id)
                relevant_partitions.append((partition_id, prob, entropy))
        
        # Sort by probability (descending)
        relevant_partitions.sort(key=lambda x: x[1], reverse=True)
        
        # If no partitions meet the threshold, return the highest probability partition
        if not relevant_partitions:
            top_partition = np.argmax(partition_probs)
            entropy = self.entropy_calculator.get_entropy(top_partition)
            relevant_partitions = [(top_partition, partition_probs[top_partition], entropy)]
        
        return relevant_partitions
    
    def add(self, element):
        """Add an element to the filter."""
        # Get the most likely partition
        relevant_partitions = self._get_relevant_partitions(element)
        partition_id = relevant_partitions[0][0]
        
        # Ensure the partition exists
        if partition_id not in self.partition_filters:
            self._initialize_partition_filter(partition_id)
        
        # Add to the appropriate filter
        self.partition_filters[partition_id].add(element)
        
        # Update entropy calculation
        element_hash = hash(element)
        self.entropy_calculator.update(partition_id, element_hash)
        
        # Update statistics
        self.partition_stats[partition_id]['items_added'] += 1
        self.total_items += 1
        
        # Check if maintenance is needed
        self.maintenance_counter += 1
        if self.maintenance_counter >= self.maintenance_interval:
            self._perform_maintenance()
            self.maintenance_counter = 0
    
    def query(self, element):
        """
        Query if an element is in the filter.
        
        Returns True if the element might be in the filter, False if definitely not.
        """
        # Get relevant partitions
        relevant_partitions = self._get_relevant_partitions(element)
        
        # Update query statistics
        self.total_queries += 1
        
        # Check each relevant partition, prioritizing by probability and entropy
        # Sort by entropy (ascending) so we check low-entropy partitions first
        # Low entropy partitions are more likely to give definitive answers
        relevant_partitions.sort(key=lambda x: x[2])
        
        for partition_id, prob, entropy in relevant_partitions:
            # Skip if partition doesn't exist
            if partition_id not in self.partition_filters:
                continue
            
            # Update partition statistics
            self.partition_stats[partition_id]['queries'] += 1
            self.partition_stats[partition_id]['last_accessed'] = time.time()
            
            # Determine how many levels to check based on entropy
            # Lower entropy means we can check more levels for better accuracy
            max_level = 3 if entropy < 3.0 else (2 if entropy < 4.0 else 1)
            
            # Check if element is in this partition's filter
            if self.partition_filters[partition_id].check(element, max_level):
                self.partition_stats[partition_id]['hits'] += 1
                return True
        
        return False
    
    def _perform_maintenance(self):
        """Perform system maintenance: merge, split, or reconfigure partitions."""
        print("\nPerforming system maintenance...")
        
        # Get current entropies
        entropies = self.entropy_calculator.get_all_entropies()
        
        # Identify candidates for splitting (low entropy, high query volume)
        split_candidates = []
        for partition_id, stats in self.partition_stats.items():
            if partition_id not in entropies:
                continue
                
            entropy = entropies[partition_id]
            queries = stats['queries']
            
            if (entropy < self.entropy_threshold_low and 
                queries > self.query_threshold and 
                stats['items_added'] > 1000):
                split_candidates.append((partition_id, entropy, queries))
        
        # Sort by entropy (ascending) and queries (descending)
        split_candidates.sort(key=lambda x: (x[1], -x[2]))
        
        # Identify candidates for merging (high entropy, low query volume)
        merge_candidates = []
        for partition_id, stats in self.partition_stats.items():
            if partition_id not in entropies:
                continue
                
            entropy = entropies[partition_id]
            queries = stats['queries']
            
            if (entropy > self.entropy_threshold_high or 
                (queries < self.query_threshold / 10 and stats['items_added'] > 0)):
                merge_candidates.append((partition_id, entropy, queries))
        
        # Sort by entropy (descending) and queries (ascending)
        merge_candidates.sort(key=lambda x: (-x[1], x[2]))
        
        # Perform splits (up to 2 per maintenance cycle)
        splits_performed = 0
        for partition_id, entropy, queries in split_candidates:
            if splits_performed >= 2:
                break
                
            print(f"Splitting partition {partition_id} (entropy={entropy:.4f}, queries={queries})")
            self._split_partition(partition_id)
            splits_performed += 1
        
        # Perform merges (up to 1 per maintenance cycle)
        if len(merge_candidates) >= 2 and splits_performed < 2:
            p1 = merge_candidates[0][0]
            p2 = merge_candidates[1][0]
            print(f"Merging partitions {p1} and {p2}")
            self._merge_partitions(p1, p2)
        
        # Print maintenance summary
        print("Maintenance complete.")
        print(f"Total partitions: {len(self.partition_filters)}")
        print(f"Total items: {self.total_items}")
        print(f"Total queries: {self.total_queries}")
    
    def _split_partition(self, partition_id):
        """Split a partition into two new partitions."""
        # This is a simplified implementation
        # In a full implementation, we would:
        # 1. Train a specialized model to split this partition
        # 2. Reassign elements to the new partitions
        # 3. Create new Bloom filters
        
        # For this implementation, we'll just create a new empty partition
        new_partition_id = max(self.partition_filters.keys()) + 1
        self._initialize_partition_filter(new_partition_id)
        
        # Update partition mapping
        self.partition_mapping[partition_id] = [partition_id, new_partition_id]
        
        # Note: In a real implementation, we would need to redistribute elements
        # between the original and new partition
    
    def _merge_partitions(self, partition_id1, partition_id2):
        """Merge two partitions into one."""
        # This is a simplified implementation
        # In a full implementation, we would:
        # 1. Create a new Bloom filter with combined capacity
        # 2. Add all elements from both partitions
        # 3. Update the model to reflect the merge
        
        # For this implementation, we'll just keep the first partition and remove the second
        if partition_id2 in self.partition_filters:
            del self.partition_filters[partition_id2]
        
        # Update partition mapping
        self.partition_mapping[partition_id2] = [partition_id1]
        
        # Note: In a real implementation, we would need to combine the Bloom filters
    
    def retrain_model(self, training_data, labels=None):
        """Retrain the partitioning model with new data."""
        # This would be similar to the initial training but would preserve
        # existing partitions and adapt the model to the current state
        # Not fully implemented in this example
        pass
    
    def get_stats(self):
        """Get statistics about the filter."""
        # Calculate total bits used by all Bloom filters
        total_bits = 0
        for partition_id, filter_obj in self.partition_filters.items():
            total_bits += filter_obj.get_size_bits()
        
        # Calculate bits per item
        bits_per_item = total_bits / self.total_items if self.total_items > 0 else 0
        
        # Get entropy statistics
        entropies = self.entropy_calculator.get_all_entropies()
        avg_entropy = sum(entropies.values()) / len(entropies) if entropies else 0
        
        # Compile partition statistics
        partition_stats = {}
        for partition_id, stats in self.partition_stats.items():
            if partition_id in self.partition_filters:
                partition_stats[partition_id] = {
                    'items': stats['items_added'],
                    'queries': stats['queries'],
                    'hits': stats['hits'],
                    'entropy': entropies.get(partition_id, 0),
                    'bits': self.partition_filters[partition_id].get_size_bits()
                }
        
        return {
            'total_items': self.total_items,
            'total_queries': self.total_queries,
            'total_bits': total_bits,
            'bits_per_item': bits_per_item,
            'num_partitions': len(self.partition_filters),
            'average_entropy': avg_entropy,
            'partition_stats': partition_stats
        }


In [ ]:
def evaluate_dynamic_multilevel_learned_bloom_filter(train_urls, train_labels, test_urls, test_labels, capacity, error_rate=0.01, initial_partitions=8):
    """
    Evaluate a Dynamic Multi-Level Learned Bloom Filter with Entropy Checking.
    
    Args:
        train_urls: URLs for training
        train_labels: Labels for training (0 for safe, 1 for unsafe)
        test_urls: URLs for testing
        test_labels: Labels for testing
        capacity: Expected number of elements
        error_rate: Desired false positive rate
        initial_partitions: Initial number of partitions
        
    Returns:
        Dictionary with evaluation results
    """
    print(f"\nEvaluating DML-LBF with {initial_partitions} initial partitions and error_rate={error_rate}")
    
    # Initialize memory and time tracking
    start_time = time.time()
    memory_before = get_memory_usage()
    print(f"Initial memory usage: {memory_before:.2f} MB")
    
    # Create and train DML-LBF
    dmlbf = DynamicMultiLevelLearnedBloomFilter(capacity, error_rate, initial_partitions)
    dmlbf.train(train_urls, train_labels)
    
    # Track memory after model creation
    memory_after_model = get_memory_usage()
    print(f"Memory after model creation: {memory_after_model:.2f} MB")
    model_creation_time = time.time() - start_time
    unsafe_train_urls = [url for url, label in zip(train_urls, train_labels) if label == 1]
    # Add all training URLs to the filter
    print(f"Adding {len(unsafe_train_urls)} URLs to the filter...")
    add_start_time = time.time()
    progress_bar = tqdm(total=len(unsafe_train_urls), desc="Adding URLs", unit="url")
    
    # Process in batches for better progress tracking
    batch_size = 1000
    for i in range(0, len(unsafe_train_urls), batch_size):
        batch_end = min(i + batch_size, len(unsafe_train_urls))
        for url in unsafe_train_urls[i:batch_end]:
            dmlbf.add(url)
        progress_bar.update(batch_end - i)
    
    progress_bar.close()
    
    add_time = time.time() - add_start_time
    memory_after_add = get_memory_usage()
    print(f"Memory after adding URLs: {memory_after_add:.2f} MB")
         # Test on the test_urls
    print(f"Testing {len(test_urls)} URLs against the filter...")
    test_start_time = time.time()

    true_positives = 0
    false_positives = 0
    true_negatives = 0
    false_negatives = 0

    progress_bar = tqdm(total=len(test_urls), desc="Testing URLs", unit="url")
    for url, label in zip(test_urls, test_labels):
        result = dmlbf.query(url)  # Predicts: unsafe (True) or safe (False)

        if result:  # Bloom filter says: possibly unsafe
            if label == 1:  # Actually unsafe
                true_positives += 1
            else:           # Actually safe
                false_positives += 1
        else:  # Bloom filter says: definitely safe
            if label == 1:  # Actually unsafe
                false_negatives += 1
            else:           # Actually safe
                true_negatives += 1
        progress_bar.update(1)
    progress_bar.close()
    test_time = time.time() - test_start_time
    print(f"Tested {len(test_urls)} URLs in {test_time:.2f} seconds")

    # Calculate metrics
    false_positive_rate = false_positives / len(test_urls) if len(test_urls) > 0 else 0
    false_negative_rate = false_negatives / len(train_urls) if len(train_urls) > 0 else 0
    # bits_per_item = filter_bits / len(train_urls) if len(train_urls) > 0 else 0

    

    # Get filter stats
    filter_stats = dmlbf.get_stats()
    
    # Evaluate adaptation capabilities
    print("Evaluating adaptation capabilities...")
    
    # Simulate data distribution change by adding more items from test set
    adaptation_sample = test_urls[:min(5000, len(test_urls))]
    adaptation_start_time = time.time()
    
    progress_bar = tqdm(total=len(adaptation_sample), desc="Testing adaptation", unit="url")
    for url in adaptation_sample:
        dmlbf.add(url)
        progress_bar.update(1)
    
    progress_bar.close()
    
    adaptation_time = time.time() - adaptation_start_time
    
    # Get updated stats after adaptation
    updated_stats = dmlbf.get_stats()
    
    # Evaluate entropy-based optimization
    print("Evaluating entropy-based optimization...")
    
    # Test query times for different entropy partitions
    entropy_values = list(filter_stats["partition_stats"].values())
    entropy_values.sort(key=lambda x: x["entropy"])
    
    low_entropy_partitions = [p for p, stats in filter_stats["partition_stats"].items() 
                             if stats["entropy"] < filter_stats["average_entropy"] * 0.7]
    high_entropy_partitions = [p for p, stats in filter_stats["partition_stats"].items() 
                              if stats["entropy"] > filter_stats["average_entropy"] * 1.3]
    
    # Test query times for low entropy partitions
    low_entropy_query_times = []
    if low_entropy_partitions:
        sample_size = min(1000, len(train_urls))
        sample_indices = np.random.choice(len(train_urls), sample_size, replace=False)
        low_entropy_start_time = time.time()
        
        for i in sample_indices:
            dmlbf.query(train_urls[i])
            
        low_entropy_query_times = (time.time() - low_entropy_start_time) / sample_size
    
    # Test query times for high entropy partitions
    high_entropy_query_times = []
    if high_entropy_partitions:
        sample_size = min(1000, len(train_urls))
        sample_indices = np.random.choice(len(train_urls), sample_size, replace=False)
        high_entropy_start_time = time.time()
        
        for i in sample_indices:
            dmlbf.query(train_urls[i])
            
        high_entropy_query_times = (time.time() - high_entropy_start_time) / sample_size
    
    # Prepare results
    results = {
        "filter_type": f"DML-LBF (partitions={initial_partitions}, error_rate={error_rate})",
        "initial_partitions": initial_partitions,
        "error_rate": error_rate,
        "capacity": capacity,
        "memory_before": memory_before,
        "memory_after_model": memory_after_model,
        "memory_after_add": memory_after_add,
        "memory_for_model": memory_after_model - memory_before,
        "memory_for_filters": filter_stats["total_bits"] / 8 / 1024 / 1024,  # Convert bits to MB
        "total_memory": (memory_after_model - memory_before) + (filter_stats["total_bits"] / 8 / 1024 / 1024),
        "model_creation_time": model_creation_time,
        "add_time": add_time,
        "true_positives": true_positives,
        "false_negatives": false_negatives,
        "true_negatives": true_negatives,
        "false_positives": false_positives,
        "false_positive_rate": false_positive_rate,
        "false_negative_rate": false_negative_rate,
        "bits_per_item": filter_stats["bits_per_item"],
        "total_bits": filter_stats["total_bits"],
        "num_partitions_before": len(filter_stats["partition_stats"]),
        "num_partitions_after": len(updated_stats["partition_stats"]),
        "adaptation_time": adaptation_time,
        "average_entropy_before": filter_stats["average_entropy"],
        "average_entropy_after": updated_stats["average_entropy"],
        "low_entropy_query_time": low_entropy_query_times,
        "high_entropy_query_time": high_entropy_query_times,
        'test_time':test_time,
        "accuracy": (true_positives + true_negatives) / len(test_urls) if len(test_urls) > 0 else 0,
        "precision": true_positives / (true_positives + false_positives) if (true_positives + false_positives) > 0 else 0,
        "recall": true_positives / (true_positives + false_negatives) if (true_positives + false_negatives) > 0 else 0,
        "f1_score": (2 * true_positives) / (2 * true_positives + false_positives + false_negatives) if (true_positives + false_positives + false_negatives) > 0 else 0,
    
    }
    
    # Print summary
    print("\nResults:")
    print(f"  False Positive Rate: {false_positive_rate:.4f} (expected: {error_rate})")
    print(f"  False Negative Rate: {false_negative_rate:.4f}")
    print(f"  Accuracy: {results['accuracy']:.4f}")
    print(f"  Precision: {results['precision']:.4f}")
    print(f"  Recall: {results['recall']:.4f}")
    print(f"  F1 Score: {results['f1_score']:.4f}")
    print(f"  Memory for model: {results['memory_for_model']:.2f} MB")
    print(f"  Memory for Bloom filters: {results['memory_for_filters']:.2f} MB")
    print(f"  Total memory (model + filters): {results['total_memory']:.2f} MB")
    print(f"  Bits per item: {results['bits_per_item']:.2f}")
    print(f"  Model creation time: {model_creation_time:.2f} seconds")
    print(f"  Add time: {add_time:.2f} seconds")
    print(f"  Test time: {results['test_time']:.2f} seconds")
    print(f"  Adaptation metrics:")
    print(f"    Initial partitions: {results['num_partitions_before']}")
    print(f"    Final partitions: {results['num_partitions_after']}")
    print(f"    Adaptation time: {adaptation_time:.2f} seconds")
    print(f"    Average entropy before: {results['average_entropy_before']:.4f}")
    print(f"    Average entropy after: {results['average_entropy_after']:.4f}")
    
    return results


## COMPARISON

In [ ]:
def compare_all_results(standard_results, plbf_results, clbf_results, dmlbf_results):
    """
    Compare and print the results of all Bloom filter implementations.
    
    Args:
        standard_results: Results from standard Bloom filter evaluation
        plbf_results: Results from PLBF evaluation
        clbf_results: Results from CLBF evaluation
        dmlbf_results: Results from DML-LBF evaluation
    """
    print("\n" + "="*80)
    print("COMPARISON OF ALL BLOOM FILTER IMPLEMENTATIONS")
    print("="*80)
    
    # Compare false positive rates
    print("\nFalse Positive Rates:")
    for result in standard_results:
        print(f"  {result['filter_type']}: {result['false_positive_rate']:.4f} (expected: {result['error_rate']})")
    for result in plbf_results:
        print(f"  {result['filter_type']}: {result['false_positive_rate']:.4f} (expected: {result['error_rate']})")
    for result in clbf_results:
        print(f"  {result['filter_type']}: {result['false_positive_rate']:.4f} (expected: {result['error_rate']})")
    for result in dmlbf_results:
        print(f"  {result['filter_type']}: {result['false_positive_rate']:.4f} (expected: {result['error_rate']})")

    # Compare false negative rates 
    print("\nFalse Negative Rates:")
    for result in standard_results:
        print(f"  {result['filter_type']}: {result['false_negative_rate']:.4f}")
    for result in plbf_results:
        print(f"  {result['filter_type']}: {result['false_negative_rate']:.4f}")
    for result in clbf_results:
        print(f"  {result['filter_type']}: {result['false_negative_rate']:.4f}")
    for result in dmlbf_results:
        print(f"  {result['filter_type']}: {result['false_negative_rate']:.4f}")
        
    # Compare Precision values 
    print("\nPrecision of different filters:")
    for result in standard_results:
        print(f"  {result['filter_type']}: {result['precision']:.4f}")
    for result in plbf_results:
        print(f"  {result['filter_type']}: {result['precision']:.4f}")
    for result in clbf_results:
        print(f"  {result['filter_type']}: {result['precision']:.4f}")
    for result in dmlbf_results:
        print(f"  {result['filter_type']}: {result['precision']:.4f}")

    # Compare recall values
    print("\nRecall of different filters:")
    for result in standard_results:
        print(f"  {result['filter_type']}: {result['recall']:.4f}")
    for result in plbf_results:
        print(f"  {result['filter_type']}: {result['recall']:.4f}")
    for result in clbf_results:
        print(f"  {result['filter_type']}: {result['recall']:.4f}")
    for result in dmlbf_results:
        print(f"  {result['filter_type']}: {result['recall']:.4f}")

    
        
    # Compare memory usage
    print("\nMemory Usage:")
    for result in standard_results:
        print(f"  {result['filter_type']}: {result['filter_size_kb']:.2f} MB")
    for result in plbf_results:
        print(f"  {result['filter_type']}: {result['memory_for_model']:.2f} MB (model) + {result['memory_for_filters']:.2f} MB (filters) = {result['total_memory']:.2f} MB (total)")
    for result in clbf_results:
        print(f"  {result['filter_type']}: {result['memory_for_model']:.2f} MB (model) + {result['memory_for_filters']:.2f} MB (filters) = {result['total_memory']:.2f} MB (total)")
    for result in dmlbf_results:
        print(f"  {result['filter_type']}: {result['memory_for_model']:.2f} MB (model) + {result['memory_for_filters']:.2f} MB (filters) = {result['total_memory']:.2f} MB (total)")
    
    # Compare bits per item
    print("\nBits per Item:")
    for result in standard_results:
        print(f"  {result['filter_type']}: {result['bits_per_item']:.2f}")
    for result in plbf_results:
        print(f"  {result['filter_type']}: {result['bits_per_item']:.2f}")
    for result in clbf_results:
        print(f"  {result['filter_type']}: {result['bits_per_item']:.2f}")
    for result in dmlbf_results:
        print(f"  {result['filter_type']}: {result['bits_per_item']:.2f}")

    # Compare accuracies
    print("\nAccuracy of different bloom filters:")
    for result in standard_results:
        print(f"  {result['filter_type']}: {result['accuracy']:.4f}")
    for result in plbf_results:
        print(f"  {result['filter_type']}: {result['accuracy']:.4f}")
    for result in clbf_results:
        print(f"  {result['filter_type']}: {result['accuracy']:.4f}")
    for result in dmlbf_results:
        print(f"  {result['filter_type']}: {result['accuracy']:.4f}")

    # Compare F1 Scores
    print("\nF1 Scores of different bloom filters:")
    for result in standard_results:
        print(f"  {result['filter_type']}: {result['f1_score']:.4f}")
    for result in plbf_results:
        print(f"  {result['filter_type']}: {result['f1_score']:.4f}")
    for result in clbf_results:
        print(f"  {result['filter_type']}: {result['f1_score']:.4f}")
    for result in dmlbf_results:
        print(f"  {result['filter_type']}: {result['f1_score']:.4f}")
    
    # Compare add times
    print("\nAdd Times:")
    for result in standard_results:
        print(f"  {result['filter_type']}: {result['add_time']:.2f} seconds")
    for result in plbf_results:
        print(f"  {result['filter_type']}: {result['add_time']:.2f} seconds (+ {result['model_creation_time']:.2f} seconds for model training)")
    for result in clbf_results:
        print(f"  {result['filter_type']}: {result['add_time']:.2f} seconds (+ {result['model_creation_time']:.2f} seconds for model training)")
    for result in dmlbf_results:
        print(f"  {result['filter_type']}: {result['add_time']:.2f} seconds (+ {result['model_creation_time']:.2f} seconds for model training)")

    # Compare query times
    # print("\nQuery Times (for both true positives and false positives):")
    # for result in standard_results:
    #     total_query_time = result['test_time_positives'] + result['test_time_negatives']
    #     print(f"  {result['filter_type']}: {total_query_time:.2f} seconds")
    # for result in plbf_results:
    #     total_query_time = result['test_time_positives'] + result['test_time_negatives']
    #     print(f"  {result['filter_type']}: {total_query_time:.2f} seconds")
    # for result in clbf_results:
    #     total_query_time = result['test_time_positives'] + result['test_time_negatives']
    #     print(f"  {result['filter_type']}: {total_query_time:.2f} seconds")
    # for result in dmlbf_results:
    #     total_query_time = result['test_time_positives'] + result['test_time_negatives']
    #     print(f"  {result['filter_type']}: {total_query_time:.2f} seconds")
    
    # Compare adaptation capabilities (only for DML-LBF)
    print("\nAdaptation Capabilities:")
    for result in dmlbf_results:
        print(f"  {result['filter_type']}:")
        print(f"    Partitions before/after adaptation: {result['num_partitions_before']}/{result['num_partitions_after']}")
        print(f"    Entropy before/after adaptation: {result['average_entropy_before']:.4f}/{result['average_entropy_after']:.4f}")
        print(f"    Adaptation time: {result['adaptation_time']:.2f} seconds")
    
    # Add summary section
    print("\nSUMMARY:")
    print("  Standard Bloom Filter advantages:")
    print("    - Simpler implementation")
    print("    - No training required")
    print("    - Faster setup time")
    print("    - Lower memory overhead for small datasets")
    
    print("\n  Partitioned Learned Bloom Filter advantages:")
    print("    - Can achieve lower false positive rates for the same memory usage")
    print("    - More efficient memory utilization for large datasets")
    print("    - Can be optimized for specific data distributions")
    print("    - Better scalability for very large datasets")
    
    print("\n  Cascaded Learned Bloom Filter advantages:")
    print("    - Fast rejection of obvious negatives in early stages")
    print("    - Optimal balance between model complexity and filter size")
    print("    - Progressive refinement through multiple stages")
    print("    - Can use different model complexities at different stages")
    
    print("\n  Dynamic Multi-Level Learned Bloom Filter advantages:")
    print("    - Entropy-aware resource allocation for optimal memory usage")
    print("    - Dynamic adaptation to changing data distributions")
    print("    - Automatic partition splitting and merging based on entropy and query patterns")
    print("    - Combines strengths of both partitioning and cascading approaches")
    print("    - Maintains performance stability under incremental workloads")
    
    print("\nRECOMMENDATION:")
    print("  - For small datasets or simple use cases: Standard Bloom Filter")
    print("  - For large datasets with uniform distribution: Partitioned Learned Bloom Filter")
    print("  - For datasets with varying difficulty of classification: Cascaded Learned Bloom Filter")
    print("  - For dynamic datasets with changing distributions: Dynamic Multi-Level Learned Bloom Filter")
    print("  - When both memory efficiency and adaptation are critical: Dynamic Multi-Level Learned Bloom Filter")

In [ ]:
def main(length_to_consider=10000, tts = 0.4):
    """Main function to run the comparison.

    Args:
        length_to_consider: Optional integer to limit the number of rows to use from the dataset.
                           If None, the entire dataset is used.
    """
    # Check if CUDA is available and print device information
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Using device: {device}")
    if device.type == "cuda":
        print(f"CUDA Device: {torch.cuda.get_device_name(0)}")
        print(f"CUDA Version: {torch.version.cuda}")
        print(f"PyTorch CUDA available: {torch.cuda.is_available()}")

    print("Loading dataset...")
    df = pd.read_csv('/kaggle/input/binary-url-dataset-csv/binary_url_dataset.csv')
    print(f"Loaded {len(df)} rows from binary_url_dataset.csv")

 
    # Stratified sampling to maintain label distribution
    

    # Check for duplicates
    print("Checking for duplicates...")
    duplicates = df.duplicated(subset=['url']).sum()
    print(f"Found {duplicates} duplicate URLs ({duplicates/len(df)*100:.2f}% of the dataset)")

    # Remove duplicates
    df = df.drop_duplicates(subset=['url'])
    print(f"Dataset after removing duplicates: {len(df)} rows")
       # Limit the dataset size if specified

    print(f"Limiting dataset to {length_to_consider} rows for faster processing")
    df = df[0:length_to_consider]
    # Convert labels to binary (0 for safe, 1 for unsafe)
    df['binary_label'] = df['label'].apply(lambda x: 1 if x == 'unsafe' else 0)
    
    safe_urls = df[df['binary_label'] == 0]
    unsafe_urls = df[df['binary_label'] == 1]

    print(f"Safe URLs: {len(safe_urls)}")
    print(f"Unsafe URLs: {len(unsafe_urls)}")

    # Train/test split within each group
    safe_train_urls, safe_test_urls = train_test_split(
        safe_urls, test_size=tts, random_state=42
    )
    unsafe_train_urls, unsafe_test_urls = train_test_split(
        unsafe_urls, test_size=tts, random_state=42
    )

    # Print stats
    print(f"\n--- Safe URLs ---")
    print(f"Train: {len(safe_train_urls)}")
    print(f"Test:  {len(safe_test_urls)}")

    print(f"\n--- Unsafe URLs ---")
    print(f"Train: {len(unsafe_train_urls)}")
    print(f"Test:  {len(unsafe_test_urls)}")

    # split the dataset into safe and unsafe URLs
    safe_urls = df[df['binary_label'] == 0]
    unsafe_urls = df[df['binary_label'] == 1]
    print(f"Safe URLs: {len(safe_urls)}")
    print(f"Unsafe URLs: {len(unsafe_urls)}")

    
    test_urls = pd.concat([unsafe_test_urls, safe_test_urls], ignore_index=True).sample(frac=1, random_state=42).reset_index(drop=True)

    # Evaluate standard Bloom filters with different error rates
    capacity_standard = len(unsafe_urls)
    standard_results = []
    for error_rate in [0.1, 0.01, 0.001]:
        result = evaluate_standard_bloom_filter(
            unsafe_urls, test_urls, capacity=capacity_standard, error_rate=error_rate
        )
        standard_results.append(result)

    # Evaluate PLBFs with different configurations
    
    plbf_results = []
    train_plbf = pd.concat([unsafe_urls, safe_train_urls], ignore_index=True).sample(frac=1, random_state=42).reset_index(drop=True)
    capacity_plbf = len(unsafe_urls)
    
    train_urls_plbf = train_plbf['url'].values
    train_labels_plbf = train_plbf['binary_label'].values
    test_urls_plbf = test_urls['url'].values
    test_labels_plbf = test_urls['binary_label'].values
    
    # Test with different error rates
    for error_rate in [0.1, 0.01, 0.001]:
        result = evaluate_partitioned_learned_bloom_filter(
            train_urls_plbf, train_labels_plbf, test_urls_plbf, test_labels_plbf,
            capacity=capacity_plbf, error_rate=error_rate, num_partitions=3
        )
        plbf_results.append(result)

    # Evaluate CLBFs with different configurations
    clbf_results = []

    # Test with different error rates
    for error_rate in [0.1, 0.01, 0.001]:
        result = evaluate_cascaded_learned_bloom_filter(
            train_urls_plbf, train_labels_plbf, test_urls_plbf, test_labels_plbf,
            capacity=capacity_plbf, error_rate=error_rate, num_stages=2
        )
        clbf_results.append(result)
        
     # Evaluate DMLBFs with different configurations
    dmlbf_results = []
    
    # Evaluate DML-LBF with different error rates
    for error_rate in [0.1, 0.01, 0.001]:
        result = evaluate_dynamic_multilevel_learned_bloom_filter(
            train_urls_plbf, train_labels_plbf, test_urls_plbf, test_labels_plbf,
            capacity=capacity_plbf, error_rate=error_rate, initial_partitions=8
        )
        dmlbf_results.append(result)

    # Compare the results
    compare_all_results(standard_results, plbf_results, clbf_results,dmlbf_results)

    print("\nComparison complete!")

if __name__ == "__main__":
    # Set the length_to_consider parameter here
    # Use None to use the entire dataset, or set a specific number (e.g., 10000) for faster processing
    length_to_consider = 10000  # Change this value as needed

    # Run main function with specified length
    main(length_to_consider=length_to_consider)